# Is it a banger?

In [110]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram

%matplotlib inline
import os

In [4]:
import glob
import tensorflow as tf
import librosa.display


KeyError: "Registering two gradient with name 'Sum' !(Previous registration was in <module> //anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:16)"

In [5]:
def plot_waveforms(audio, log_specgram):
    # audio is np.array holding sample values
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    librosa.display.waveplot(audio, sr=22050)
    plt.subplot(1, 2, 2)
    librosa.display.specshow(log_specgram, x_axis='time',y_axis='log')
    plt.colorbar(format='%+2.0f dB')

In [89]:
sess = tf.InteractiveSession()
audio_array = audio.eval()

In [3]:
training_data_glob = '../data/train/*.mp3'

tf.reset_default_graph()
with tf.Graph().as_default():
    # take files one by one and read data from them
    files = glob.glob(training_data_glob)    
    queue = tf.train.string_input_producer(files, num_epochs=1)
    fname = queue.dequeue()
    audio = load(fname)
    audio = preprocess(audio)
    samples = tf.train.slice_input_producer([audio], num_epochs=1)
    batch = tf.train.batch(samples, 10)

    model = tf.identity(batch) # do nothing

    init = [tf.global_variables_initializer(), tf.local_variables_initializer()]

    coord = tf.train.Coordinator()

    with tf.Session() as session:
        session.run(init)
        threads = tf.train.start_queue_runners(sess=session, coord=coord)
        for _ in range(10):
            try:
                result = session.run(model)
                print(batch)
            except tf.errors.OutOfRangeError:
                print('Error')
                coord.request_stop()
        coord.request_stop()
        coord.join(threads)

NameError: name 'load' is not defined

In [8]:
training_data_glob = '../data/train/*.mp3'

tf.reset_default_graph()
with tf.Graph().as_default():
    # take files one by one and read data from them
    files = glob.glob(training_data_glob)    
    queue = tf.train.string_input_producer(files, num_epochs=None)
    fname = queue.dequeue()
    audio = load(fname)
    audio = preprocess(audio)
    samples = tf.train.slice_input_producer([audio], num_epochs=None)
    batch = tf.train.batch(samples, 10)

    model = tf.identity(batch) # do nothing

    init = [tf.global_variables_initializer(), tf.local_variables_initializer()]

    coord = tf.train.Coordinator()

    with tf.Session() as session:
        session.run(init)
        threads = tf.train.start_queue_runners(sess=session, coord=coord)
        arr = samples[0].eval()
        for _ in range(10):
            try:
                result = session.run(model)
                print(batch.eval())
            except tf.errors.OutOfRangeError:
                print('OutOfRangeError')
                coord.request_stop()
        coord.request_stop()
        coord.join(threads)

[[[ 0.47797358  0.40630198]
  [ 0.48043031  0.40489814]
  [ 0.4811475   0.39575797]
  ..., 
  [ 0.35785457  0.41376364]
  [ 0.41062027  0.43729305]
  [ 0.39499503  0.40543219]]

 [[ 0.44512093  0.24942397]
  [ 0.48648813  0.28505379]
  [ 0.49127945  0.23540093]
  ..., 
  [ 0.51633477  0.32217899]
  [ 0.50725567  0.32271305]
  [ 0.50139624  0.32712293]]

 [[ 0.49936673  0.50225067]
  [ 0.49828336  0.50070953]
  [ 0.50402075  0.50412756]
  ..., 
  [ 0.50508893  0.50533307]
  [ 0.50016022  0.50055695]
  [ 0.49993134  0.49976349]]

 ..., 
 [[ 0.3534905   0.34949264]
  [ 0.35812923  0.35541314]
  [ 0.36244753  0.3610895 ]
  ..., 
  [ 0.56211185  0.5615778 ]
  [ 0.55994511  0.55982304]
  [ 0.55858701  0.55890745]]

 [[ 0.35841915  0.42154574]
  [ 0.39537653  0.44013125]
  [ 0.35655755  0.37100786]
  ..., 
  [ 0.44679943  0.53308922]
  [ 0.48174256  0.53519493]
  [ 0.50312048  0.52193481]]

 [[ 0.60822457  0.59905398]
  [ 0.60720229  0.60109866]
  [ 0.65818268  0.64551765]
  ..., 
  [ 0.55088

[[[ 0.59156179  0.60120547]
  [ 0.62175936  0.63686579]
  [ 0.64048219  0.65828943]
  ..., 
  [ 0.59472036  0.67354852]
  [ 0.60555428  0.68174255]
  [ 0.6322118   0.70086217]]

 [[ 0.59974056  0.72066838]
  [ 0.59465933  0.74689859]
  [ 0.56395823  0.74685282]
  ..., 
  [ 0.34219882  0.31046006]
  [ 0.25171283  0.19768062]
  [ 0.24812695  0.18133822]]

 [[ 0.54493016  0.45105669]
  [ 0.59038681  0.49610132]
  [ 0.61680019  0.52317083]
  ..., 
  [ 0.28056765  0.29605553]
  [ 0.26257727  0.26845199]
  [ 0.2728008   0.27078661]]

 ..., 
 [[ 0.49839017  0.49924466]
  [ 0.50287628  0.5022049 ]
  [ 0.50409704  0.50243378]
  ..., 
  [ 0.50890368  0.50228119]
  [ 0.49932095  0.49306479]
  [ 0.49668118  0.49137101]]

 [[ 0.70003814  0.78892195]
  [ 0.72376591  0.77933925]
  [ 0.74985886  0.77926302]
  ..., 
  [ 0.64460212  0.54105437]
  [ 0.62990767  0.5162127 ]
  [ 0.63526356  0.50807965]]

 [[ 0.33748379  0.34071872]
  [ 0.36882582  0.36446172]
  [ 0.43677422  0.44057372]
  ..., 
  [ 0.60479